# Estimator Primitive - Code Laboratory

**Section 6: Estimator & VQE/QAOA** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

| Method | Signature | Returns | Use When |
|--------|-----------|---------|----------|
| `StatevectorEstimator()` | `StatevectorEstimator()` | Estimator instance | Local simulation |
| `estimator.run()` | `run([(qc, obs, params, prec)])` | `PrimitiveJob` | Calculate ⟨O⟩ |
| `SparsePauliOp()` | `SparsePauliOp(paulis, coeffs)` | Observable | Define Hamiltonian |
| `result[i].data.evs` | - | `float` | Get expectation value |
| `result[i].data.stds` | - | `float` | Get standard deviation |

---

In [ ]:
# ============================================================
# ENVIRONMENT SETUP - Run this first!
# ============================================================
"""
Qiskit Code Laboratory - Estimator Primitive
Prerequisites: See README.md for conceptual background
"""

# Standard imports
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# =============================================================
# UTILITY FUNCTIONS FOR THIS NOTEBOOK
# =============================================================

def run_and_print(estimator, pubs, label=""):
    """Run estimator and print formatted results."""
    job = estimator.run(pubs)
    result = job.result()
    if label:
        print(f"\n{label}")
        print("-" * 40)
    for i in range(len(pubs)):
        evs = result[i].data.evs
        stds = result[i].data.stds
        print(f"  PUB[{i}]: ⟨O⟩ = {evs:.4f} ± {stds:.4f}")
    return result

def verify_expectation(estimator, qc, obs, expected, tolerance=0.01, label=""):
    """Verify expectation value matches expected."""
    job = estimator.run([(qc, obs)])
    actual = job.result()[0].data.evs
    passed = abs(actual - expected) < tolerance
    status = "✅" if passed else "❌"
    print(f"{status} {label}: Expected {expected:.4f}, Got {actual:.4f}")
    return passed

print("✅ Environment ready")
print("   StatevectorEstimator for local simulation")
print("   SparsePauliOp for observables")

In [ ]:
---

## 1️⃣ `StatevectorEstimator` - Basic Usage

### API Signature
```python
from qiskit.primitives import StatevectorEstimator
estimator = StatevectorEstimator()
job = estimator.run(pubs)  # PUB = (circuit, observable, params?, precision?)
result = job.result()
expectation = result[0].data.evs  # ⚠️ PLURAL: evs not ev!
```

### PUB Format (MEMORIZE!)
| Component | Required | Type | Description |
|-----------|----------|------|-------------|
| `circuit` | ✅ YES | QuantumCircuit | **NO measurements!** |
| `observable` | ✅ YES | SparsePauliOp | What to measure |
| `parameter_values` | Optional | list | Values for Parameters |
| `precision` | Optional | float | Target precision |

In [ ]:
# BASIC USAGE: StatevectorEstimator
# =================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# Create Bell state (NO measurements!)
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
# NOTE: No qc.measure() for Estimator!

# Define observable
obs = SparsePauliOp('ZZ')

# Create estimator and run
estimator = StatevectorEstimator()
job = estimator.run([(qc, obs)])  # Note: list of tuples!
result = job.result()

# Access result - MEMORIZE THIS PATTERN!
expectation = result[0].data.evs  # ⚠️ evs (plural)!
std_dev = result[0].data.stds     # ⚠️ stds (plural)!

print("Bell State ⟨ZZ⟩ Expectation:")
print(f"  result[0].data.evs  = {expectation:.4f}")
print(f"  result[0].data.stds = {std_dev:.4f}")
print()
print("📝 Result access: result[i].data.evs (plural!)")
print("   i = index in PUB list")

In [ ]:
# 🚨 TRAP DEMONSTRATION: No Measurements for Estimator!
# =====================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# This works - NO measurements
qc_good = QuantumCircuit(2)
qc_good.h(0)
qc_good.cx(0, 1)
# Good: No measurements!

# This FAILS - has measurements
qc_bad = QuantumCircuit(2, 2)
qc_bad.h(0)
qc_bad.cx(0, 1)
qc_bad.measure_all()  # ❌ WRONG!

obs = SparsePauliOp('ZZ')
estimator = StatevectorEstimator()

print("Testing Estimator with measurements...")
print("=" * 50)

# Test good circuit
try:
    job = estimator.run([(qc_good, obs)])
    result = job.result()[0].data.evs
    print(f"✅ Without measurements: ⟨ZZ⟩ = {result:.4f}")
except Exception as e:
    print(f"❌ Error: {e}")

# Test bad circuit
try:
    job = estimator.run([(qc_bad, obs)])
    result = job.result()[0].data.evs
    print(f"✅ With measurements: ⟨ZZ⟩ = {result:.4f}")
except Exception as e:
    print(f"❌ With measurements: ERROR!")
    print(f"   {type(e).__name__}")

print()
print("🎯 EXAM TRAP: Sampler NEEDS measurements, Estimator does NOT!")

In [ ]:
---

## 2️⃣ `SparsePauliOp` - Observable Construction

### API Signature
```python
from qiskit.quantum_info import SparsePauliOp

# Single Pauli
Z = SparsePauliOp('Z')

# Multi-qubit (⚠️ RIGHT-TO-LEFT ordering!)
ZX = SparsePauliOp('ZX')  # Z on q1, X on q0

# Hamiltonian with coefficients
H = SparsePauliOp(['ZZ', 'XX'], coeffs=[0.5, 0.3])
```

### ⚠️ CRITICAL: Qubit Ordering (RIGHT TO LEFT!)
```
SparsePauliOp('ZX')
     ↑↑
     │└── X on qubit 0
     └─── Z on qubit 1
```

In [ ]:
# SparsePauliOp Construction Examples
# ===================================

from qiskit.quantum_info import SparsePauliOp

# Single Pauli operators
print("1. Single Pauli Operators:")
Z = SparsePauliOp('Z')
X = SparsePauliOp('X')
Y = SparsePauliOp('Y')
print(f"   Z = {Z}")
print(f"   X = {X}")

# Multi-qubit operators (RIGHT TO LEFT ordering!)
print("\n2. Multi-qubit Operators (tensor product order):")
ZZ = SparsePauliOp('ZZ')  # Z⊗Z
ZX = SparsePauliOp('ZX')  # Z on q1, X on q0
ZI = SparsePauliOp('ZI')  # Z on q1, I on q0
print(f"   'ZZ' = {ZZ}")
print(f"   'ZX' = Z on q1, X on q0")
print(f"   'ZI' = Z on q1, I on q0")

# Hamiltonian with coefficients
print("\n3. Hamiltonian Construction (H = 0.5*ZZ + 0.3*XX):")

# Method 1: List of Paulis + coeffs array
H1 = SparsePauliOp(['ZZ', 'XX'], coeffs=[0.5, 0.3])
print(f"   Method 1: SparsePauliOp(['ZZ', 'XX'], coeffs=[0.5, 0.3])")

# Method 2: from_list with tuples
H2 = SparsePauliOp.from_list([('ZZ', 0.5), ('XX', 0.3)])
print(f"   Method 2: SparsePauliOp.from_list([('ZZ', 0.5), ('XX', 0.3)])")

# Method 3: Operator arithmetic
H3 = 0.5 * SparsePauliOp('ZZ') + 0.3 * SparsePauliOp('XX')
print(f"   Method 3: 0.5 * SparsePauliOp('ZZ') + 0.3 * SparsePauliOp('XX')")

print(f"\n   H = {H1}")

In [ ]:
# 🚨 TRAP DEMONSTRATION: Qubit Ordering
# =====================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# Create circuit: X on qubit 0 only (q0=1, q1=0 → |01⟩)
qc = QuantumCircuit(2)
qc.x(0)  # Only qubit 0 is |1⟩

estimator = StatevectorEstimator()

# Test 'ZI' vs 'IZ' - which measures qubit 0?
ZI = SparsePauliOp('ZI')  # Z on q1, I on q0
IZ = SparsePauliOp('IZ')  # I on q1, Z on q0

job = estimator.run([(qc, ZI), (qc, IZ)])
result = job.result()

print("Circuit: X on qubit 0 → state |01⟩")
print("=" * 45)
print(f"⟨ZI⟩ = {result[0].data.evs:.1f}  (Z on q1 = +1, q1 is |0⟩)")
print(f"⟨IZ⟩ = {result[1].data.evs:.1f}  (Z on q0 = -1, q0 is |1⟩)")
print()
print("📝 MEMORIZE: 'ZI' = Z⊗I = Z on qubit 1, I on qubit 0")
print("   String reads RIGHT to LEFT for qubits!")
print()
print("🎯 EXAM TRAP: 'ZI' does NOT mean Z on qubit 0!")

In [ ]:
---

## 3️⃣ PUB Formats - All Variations

### PUB = (circuit, observable, parameter_values, precision)

| Scenario | Format | Example |
|----------|--------|---------|
| Basic | `(qc, obs)` | `estimator.run([(qc, obs)])` |
| With params | `(qc, obs, params)` | `estimator.run([(qc, obs, [0.5])])` |
| With precision | `(qc, obs, None, prec)` | `estimator.run([(qc, obs, None, 0.01)])` |
| All options | `(qc, obs, params, prec)` | `estimator.run([(qc, obs, [0.5], 0.01)])` |

In [ ]:
# PUB Format Examples - All Variations
# ====================================

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# Setup
qc_fixed = QuantumCircuit(2)
qc_fixed.h(0)
qc_fixed.cx(0, 1)

theta = Parameter('θ')
qc_param = QuantumCircuit(1)
qc_param.ry(theta, 0)

obs = SparsePauliOp('ZZ')
obs_single = SparsePauliOp('Z')
estimator = StatevectorEstimator()

print("PUB FORMAT EXAMPLES")
print("=" * 55)

# Format 1: Basic (circuit, observable)
print("\n[1] Basic: (circuit, observable)")
job = estimator.run([(qc_fixed, obs)])
print(f"    ⟨ZZ⟩ = {job.result()[0].data.evs:.4f}")

# Format 2: With parameters (circuit, observable, params)
print("\n[2] With params: (circuit, observable, [values])")
job = estimator.run([(qc_param, obs_single, [1.57])])
print(f"    θ=π/2: ⟨Z⟩ = {job.result()[0].data.evs:.4f}")

# Format 3: Multiple parameter sets (batch)
print("\n[3] Batch: Multiple parameter values")
theta_vals = [[0.0], [1.57], [3.14]]
pubs = [(qc_param, obs_single, val) for val in theta_vals]
job = estimator.run(pubs)
for i, val in enumerate(theta_vals):
    print(f"    θ={val[0]:.2f}: ⟨Z⟩ = {job.result()[i].data.evs:.4f}")

# Format 4: Multiple observables
print("\n[4] Multiple observables (same circuit)")
observables = [SparsePauliOp('ZZ'), SparsePauliOp('XX'), SparsePauliOp('YY')]
pubs = [(qc_fixed, obs) for obs in observables]
job = estimator.run(pubs)
for i, label in enumerate(['ZZ', 'XX', 'YY']):
    print(f"    ⟨{label}⟩ = {job.result()[i].data.evs:.4f}")

In [ ]:
---

## 4️⃣ Bell State Expectation Values (MEMORIZE!)

### Reference Values
```
Bell |Φ+⟩ = (|00⟩ + |11⟩)/√2

⟨ZZ⟩ = +1.0  ← Correlated in Z
⟨XX⟩ = +1.0  ← Correlated in X
⟨YY⟩ = -1.0  ← ANTI-correlated in Y (⚠️ negative!)
⟨ZI⟩ = 0.0   ← Single qubit maximally mixed
⟨IZ⟩ = 0.0   ← Single qubit maximally mixed
```

In [ ]:
# Bell State Expectation Values - Verify Reference
# =================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# Create Bell state |Φ+⟩ = (|00⟩ + |11⟩)/√2
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# All observables to test
observables = {
    'ZZ': SparsePauliOp('ZZ'),
    'XX': SparsePauliOp('XX'),
    'YY': SparsePauliOp('YY'),
    'ZI': SparsePauliOp('ZI'),
    'IZ': SparsePauliOp('IZ'),
}

estimator = StatevectorEstimator()
pubs = [(qc, obs) for obs in observables.values()]
job = estimator.run(pubs)
result = job.result()

print("Bell State |Φ+⟩ Expectation Values:")
print("=" * 40)
expected = {'ZZ': 1.0, 'XX': 1.0, 'YY': -1.0, 'ZI': 0.0, 'IZ': 0.0}
for i, (name, exp) in enumerate(expected.items()):
    actual = result[i].data.evs
    status = "✅" if abs(actual - exp) < 0.01 else "❌"
    print(f"⟨{name}⟩ = {actual:+.1f}  (expected {exp:+.1f}) {status}")

print()
print("🎯 MEMORIZE: Bell ⟨YY⟩ = -1 (anti-correlated!)")

In [ ]:
---

## 5️⃣ Hamiltonian Expectation Values

### Common Hamiltonians
```python
# Ising: H = -J*ZZ - h*Z
H = SparsePauliOp(['ZZ', 'ZI'], coeffs=[-J, -h])

# Heisenberg: H = XX + YY + ZZ
H = SparsePauliOp(['XX', 'YY', 'ZZ'], coeffs=[1, 1, 1])

# H2 molecule (5 terms)
H = SparsePauliOp(['II', 'ZI', 'IZ', 'ZZ', 'XX'],
                  coeffs=[-1.05, 0.39, 0.39, -0.01, 0.18])
```

In [ ]:
# Hamiltonian Energy Calculation
# ==============================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# H2 Molecule Hamiltonian (Jordan-Wigner encoding)
H2 = SparsePauliOp(
    ["II", "ZI", "IZ", "ZZ", "XX"],
    [-1.05, 0.39, 0.39, -0.01, 0.18]
)

print("H2 Molecule Hamiltonian:")
print(f"  H = -1.05*II + 0.39*ZI + 0.39*IZ - 0.01*ZZ + 0.18*XX")
print(f"  Number of terms: {len(H2)}")

# Test different states
estimator = StatevectorEstimator()

# State 1: |00⟩ (both orbitals empty)
qc1 = QuantumCircuit(2)
job = estimator.run([(qc1, H2)])
E1 = job.result()[0].data.evs

# State 2: |11⟩ (both orbitals occupied)
qc2 = QuantumCircuit(2)
qc2.x([0, 1])
job = estimator.run([(qc2, H2)])
E2 = job.result()[0].data.evs

# State 3: Bell state (superposition)
qc3 = QuantumCircuit(2)
qc3.h(0)
qc3.cx(0, 1)
job = estimator.run([(qc3, H2)])
E3 = job.result()[0].data.evs

print(f"\nEnergy Calculations:")
print(f"  |00⟩ state: E = {E1:.4f} Hartree")
print(f"  |11⟩ state: E = {E2:.4f} Hartree")
print(f"  Bell state: E = {E3:.4f} Hartree")
print()
print("💡 VQE would optimize parameters to find minimum energy")

---

## 6️⃣ Parameterized Circuits with Estimator

### Pattern for VQE-style Parameter Binding
```python
theta = Parameter('θ')
qc = QuantumCircuit(1)
qc.ry(theta, 0)

# Method 1: assign_parameters then run
qc_bound = qc.assign_parameters({theta: 0.5})
job = estimator.run([(qc_bound, obs)])

# Method 2: Pass parameters in PUB (preferred)
job = estimator.run([(qc, obs, [0.5])])
```

In [ ]:
# Parameterized Circuits - RY Rotation
# =====================================

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
import numpy as np

# Create parameterized circuit
theta = Parameter('θ')
qc = QuantumCircuit(1)
qc.ry(theta, 0)

# Observable
Z_obs = SparsePauliOp('Z')
estimator = StatevectorEstimator()

# Measure ⟨Z⟩ at different angles
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi]

print("RY(θ)|0⟩ : Expectation ⟨Z⟩ = cos(θ)")
print("=" * 45)
print("θ (rad)    ⟨Z⟩      cos(θ)   Match?")
print("-" * 45)

pubs = [(qc, Z_obs, [a]) for a in angles]
job = estimator.run(pubs)
result = job.result()

for i, angle in enumerate(angles):
    ev = result[i].data.evs
    expected = np.cos(angle)
    status = "✅" if abs(ev - expected) < 0.01 else "❌"
    print(f"{angle:6.3f}    {ev:7.4f}   {expected:7.4f}   {status}")

print()
print("📝 Key: Pass parameters in PUB tuple for efficiency")

---

## 7️⃣ GHZ State Verification (Exam Pattern)

### GHZ State: |000⟩ + |111⟩)/√2
```
⟨XXX⟩ = -1  (or +1 depending on phase)
⟨ZZI⟩ = +1  (pairwise Z correlation)
⟨ZIZ⟩ = +1
⟨IZZ⟩ = +1
```

In [ ]:
# GHZ State Verification
# ======================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# Create GHZ state: (|000⟩ + |111⟩)/√2
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)

# Witnesses for genuine tripartite entanglement
witnesses = {
    'XXX': SparsePauliOp('XXX'),
    'ZZI': SparsePauliOp('ZZI'),
    'ZIZ': SparsePauliOp('ZIZ'),
    'IZZ': SparsePauliOp('IZZ'),
}

estimator = StatevectorEstimator()
pubs = [(qc, w) for w in witnesses.values()]
job = estimator.run(pubs)
result = job.result()

print("GHZ State |000⟩ + |111⟩)/√2 Witnesses:")
print("=" * 45)
expected = {'XXX': -1.0, 'ZZI': 1.0, 'ZIZ': 1.0, 'IZZ': 1.0}
for i, (name, exp) in enumerate(expected.items()):
    actual = result[i].data.evs
    status = "✅" if abs(actual - exp) < 0.01 else "❌"
    print(f"⟨{name}⟩ = {actual:+.1f}  (expected {exp:+.1f}) {status}")

# Verify genuine tripartite entanglement
xxx = result[0].data.evs
zz_sum = sum(result[i].data.evs for i in [1, 2, 3])

print()
if abs(xxx + 1.0) < 0.01 and abs(zz_sum - 3.0) < 0.01:
    print("✅ Genuine 3-qubit entanglement verified!")
else:
    print("❌ Not a valid GHZ state")

In [ ]:
---

## 8️⃣ Runtime Estimator Pattern (Real Hardware)

### Key Differences from StatevectorEstimator
```python
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# 1. Get backend
service = QiskitRuntimeService()
backend = service.least_busy(simulator=False)

# 2. Transpile circuit
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
qc_transpiled = pm.run(qc)

# 3. Apply layout to observable (!!)
obs_mapped = observable.apply_layout(qc_transpiled.layout)

# 4. Run with Estimator
estimator = EstimatorV2(mode=backend)
job = estimator.run([(qc_transpiled, obs_mapped)])
result = job.result()[0].data.evs  # Same access pattern!
```

In [ ]:
# Runtime Estimator Pattern (Reference Code)
# ==========================================

# This is reference code for real hardware - requires IBM Quantum account
print("Runtime Estimator Pattern for Real Hardware")
print("=" * 55)

code = '''
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Setup service
service = QiskitRuntimeService()
backend = service.least_busy(simulator=False, operational=True)

# Create circuit (NO measurements)
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Define observable
observable = SparsePauliOp('ZZ')

# Transpile for target backend
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
qc_transpiled = pm.run(qc)

# ⚠️ CRITICAL: Apply layout to observable!
observable_mapped = observable.apply_layout(qc_transpiled.layout)

# Run Estimator
estimator = Estimator(mode=backend)
job = estimator.run([(qc_transpiled, observable_mapped)])
result = job.result()
expectation = result[0].data.evs  # Same access pattern!
'''

print(code)
print()
print("🎯 KEY DIFFERENCES from StatevectorEstimator:")
print("  1. Import from qiskit_ibm_runtime (not qiskit.primitives)")
print("  2. Need backend from QiskitRuntimeService")
print("  3. Must transpile circuit before running")
print("  4. Must apply_layout() to observable! ⚠️")
print("  5. Same result access: result[0].data.evs")

In [ ]:
---

## 9️⃣ Resilience Options & ZNE (EXAM CRITICAL!)

### Resilience Levels
| Level | Techniques | Description |
|-------|------------|-------------|
| **0** | None | No mitigation (fastest) |
| **1** | M3 + Twirling | Balanced (default) |
| **2** | ZNE + PEC | Maximum accuracy (slowest) |

### ZNE Configuration
```python
estimator.options.resilience_level = 2
estimator.options.resilience.zne.noise_factors = (1, 3, 5)
estimator.options.resilience.zne.extrapolator = "exponential"
```

In [ ]:
# Resilience Options Configuration (Reference)
# =============================================

print("Resilience Options for Runtime Estimator")
print("=" * 50)

print("""
from qiskit_ibm_runtime import EstimatorV2 as Estimator

estimator = Estimator(mode=backend)

# Option 1: Set resilience level (0, 1, or 2)
estimator.options.resilience_level = 2  # Enable ZNE + PEC

# Option 2: Fine-grained ZNE control
estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.noise_factors = (1, 3, 5)
estimator.options.resilience.zne.extrapolator = "exponential"
estimator.options.resilience.zne.amplifier = "gate_folding"
""")

print("Resilience Levels Explained:")
print("-" * 40)
print("  Level 0: No mitigation (fastest, noisiest)")
print("  Level 1: M3 + Twirling (balanced default)")
print("  Level 2: ZNE + PEC (slowest, most accurate)")
print()
print("ZNE Options (resilience.zne.*):")
print("-" * 40)
print("  noise_factors:  (1, 3, 5) - noise amplification levels")
print("  extrapolator:   'linear', 'exponential', 'polynomial'")
print("  amplifier:      'gate_folding', 'local_folding'")
print()
print("🎯 EXAM TIP: ZNE trades runtime for accuracy")

---

## 🎯 CODE CHALLENGES

### Challenge 1: Result Access Trap
Fix the code to correctly access expectation values.

In [ ]:
# CHALLENGE 1: Fix the result access
# ===================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

qc = QuantumCircuit(1)
qc.h(0)

obs = SparsePauliOp('X')
estimator = StatevectorEstimator()
job = estimator.run([(qc, obs)])
result = job.result()

# ❌ WRONG - Fix this line!
# expectation = result.data.ev  # What's wrong here?

# ✅ YOUR FIX HERE:
expectation = result[0].data.evs  # Fix: [0] index + plural evs

# Verification
assert abs(expectation - 1.0) < 0.01, "Expected ⟨X⟩ = 1 for |+⟩ state"
print(f"✅ Challenge 1 PASSED: ⟨X⟩ = {expectation:.4f}")
print("   Key: result[0].data.evs (index + plural!)")

In [ ]:
# CHALLENGE 2: Fix the qubit ordering
# ====================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# Circuit: X on qubit 0, nothing on qubit 1
# State: |01⟩ (qubit 0 = |1⟩, qubit 1 = |0⟩)
qc = QuantumCircuit(2)
qc.x(0)

estimator = StatevectorEstimator()

# Goal: Measure Z on qubit 0 (should be -1)
# ❌ WRONG - this measures qubit 1!
# obs_wrong = SparsePauliOp('ZI')

# ✅ YOUR FIX HERE:
obs_correct = SparsePauliOp('IZ')  # I on q1, Z on q0

job = estimator.run([(qc, obs_correct)])
result = job.result()[0].data.evs

# Verification
assert abs(result - (-1.0)) < 0.01, "Expected ⟨Z_q0⟩ = -1"
print(f"✅ Challenge 2 PASSED: ⟨Z on q0⟩ = {result:.1f}")
print("   Key: 'IZ' = I⊗Z = I on q1, Z on q0 (right-to-left!)")

In [ ]:
# CHALLENGE 3: Complete the Hamiltonian calculation
# ==================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# Create Ising Hamiltonian: H = -ZZ - 0.5*XI - 0.5*IX
H = SparsePauliOp(['ZZ', 'XI', 'IX'], coeffs=[-1.0, -0.5, -0.5])

# Create trial state: |++⟩
qc = QuantumCircuit(2)
qc.h([0, 1])

# YOUR CODE: Calculate energy expectation value
estimator = StatevectorEstimator()
job = estimator.run([(qc, H)])
energy = job.result()[0].data.evs

# For |++⟩:
# ⟨ZZ⟩ = 0 (uncorrelated), ⟨XI⟩ = 1, ⟨IX⟩ = 1
# Expected: -1*0 - 0.5*1 - 0.5*1 = -1.0
expected = -1.0

# Verification
assert abs(energy - expected) < 0.01, f"Expected {expected}, got {energy}"
print(f"✅ Challenge 3 PASSED: ⟨H⟩ = {energy:.4f}")
print(f"   Breakdown: -1*⟨ZZ⟩ - 0.5*⟨XI⟩ - 0.5*⟨IX⟩ = -1*0 - 0.5*1 - 0.5*1 = -1.0")

---

## ✅ Key Takeaways

### 🧠 Mnemonics
- **ENM**: `evs` Not `ev`, Must be plural!
- **0-D-EV-S**: `result[0].data.evs` access pattern

### 🎯 Exam Critical Facts
1. **No measurements** in circuits for Estimator
2. **Qubit ordering**: `'ZX'` = Z on q1, X on q0 (right-to-left)
3. **Result access**: `result[i].data.evs` (plural!)
4. **Bell ⟨YY⟩ = -1** (anti-correlated)
5. **apply_layout()** required for Runtime Estimator

### ⚠️ Common Traps
| Trap | Wrong | Correct |
|------|-------|---------|
| Plural | `.data.ev` | `.data.evs` |
| Index | `result.data` | `result[0].data` |
| Measurements | Add `measure()` | No measurements! |
| Qubit order | `'ZI'` = Z on q0 | `'ZI'` = Z on q1 |

**Next**: [vqe_pattern.ipynb](./vqe_pattern.ipynb) - VQE and QAOA patterns